In [1]:
import pandas as pd

In [2]:
from selenium import webdriver

In [3]:
from selenium.webdriver.common.keys import Keys

In [141]:
df = pd.DataFrame(columns=['id','message','date'])
df.head()

,id,message,date


In [4]:
chromedriver = '/usr/local/bin/chromedriver'

In [5]:
driver = webdriver.Chrome(chromedriver)

select = driver.find_elements_by_css_selector
select_one = driver.find_element_by_css_selector


In [134]:
driver.get('https://www.linkedin.com/')

In [152]:
#if on the main linked in with the sign in on top right
try:
    sign_in = select_one('a.nav__button-secondary')
    sign_in.click()
    input_name = select_one('input#username')
    input_name.send_keys('keri@multerer.com')
    input_pwd = select_one('input#password')
    input_pwd.send_keys('jfdkfjadluivjdkk5')
    login_action = select_one('div.login__form_action_container ')
    login_action.click()
except:
    email_input = select_one('input.login-email')
    email_input.click()
    email_input.send_keys('keri@multerer.com')
    password_input = select_one('input.login-password')
    password_input.click()
    password_input.send_keys('jfdkfjadluivjdkk5')
    submit = select_one('#login-submit')
    submit.click()
except:
    driver.get(sign_in.get_attribute('href'))
    input_name = select_one('input#username')
    input_name.click()
    input_name.send_keys('keri@multerer.com')
    input_password = select_one('input#password')
    input_password.click()
    input_password.send_keys('jfdkfjadluivjdkk5')
    login_btn = select_one('button.btn__primary--large.from__button--floating')
    login_btn.click()
except:
    print('nope, didnt work')


SyntaxError: invalid syntax (<ipython-input-152-6be980c39c47>, line 11)

# Getting list of conversations on the left and then clicking through them to capture the full message thread

In [145]:
#accessing the messages once logged in
message_btn = select_one('span#messaging-tab-icon')
message_btn.click()

In [146]:
convo_list = select('li.msg-conversation-listitem')

In [147]:
for convo in convo_list:
    convo.click()
    try:
        message = select_one('div.msg-s-message-list-container').text
        message_list = message.split('\n')
        name = message_list[4]
        date = message_list[1]
        df = df.append({'id' : name , 'message' : message, 'date': date} , ignore_index=True)
    except:
        df = df.append({'id': '' , 'message' : 'advertisement', 'date': ''}, ignore_index=True)

In [149]:
df.head()

,id,message,date
0,Technologist,Faisal Zahid\nStatus is reachable\nFaisal Zahi...,Status is reachable
1,Technologist,Faisal Zahid\nStatus is reachable\nFaisal Zahi...,Status is reachable
2,,advertisement,
3,Andrew Yin,Recruiter\nFeb 22\nAndrew Yin sent the followi...,Feb 22
4,FEB 24,Yiran Sherry\nYiran Sherry\n· 1st\nEnterprise ...,Yiran Sherry


# Getting the stats for visits

In [193]:
#go to profile page
profile = select_one('li#profile-nav-item')
profile.click()
dropdown = select_one('ul#nav-settings__dropdown-options')
url = dropdown.find_element_by_css_selector('a')
url.get_attribute('href')
driver.get(url.get_attribute('href'))

In [194]:
#get the profile view information
views = select_one('a.profile-views')
views.get_attribute('href')
driver.get(views.get_attribute('href'))

In [195]:
#number of views in the last 90 days
views = select_one('span.me-wvmp-views__90-days-views')
views.text

'17'

In [196]:
#access all views page
all_views = select_one('li.ember-view a')
driver.get(all_views.get_attribute('href'))

In [56]:
#this was on the page when linked in asked if my info was correct
info_correct = select_one('button.primary-action')
info_correct.click()

In [ ]:
message = select_one('p.msg-spinmail-thread__message-body t-14 t-black t-normal msg-spinmail-thread__message-body--with-spacing')